In [ ]:
# %%

from langchain_community.document_loaders.pdf import PyPDFLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.embeddings.ollama import OllamaEmbeddings
from langchain_community.vectorstores.chroma import Chroma

doc_path = "data/oxford-handbook-of-cardiology-2nbsped-9780199643219_compress.pdf"
ollama_url = "http://192.168.0.100:11434"
base_url = "http://192.168.0.100:8000/v1"
chroma_path = "chroma_db/diad_man"

loader = PyPDFLoader(doc_path)
docs = loader.load()

text_splitter = RecursiveCharacterTextSplitter(chunk_size=1500, chunk_overlap=100)
texts = text_splitter.split_documents(docs)

ollama_embeddings = OllamaEmbeddings(model="mxbai-embed-large", base_url=ollama_url)

Multiple definitions in dictionary at byte 0x40 for key /Length
Multiple definitions in dictionary at byte 0x53 for key /Filter
Multiple definitions in dictionary at byte 0x132 for key /Length
Multiple definitions in dictionary at byte 0x13b for key /Name
Multiple definitions in dictionary at byte 0x149 for key /Subtype
Multiple definitions in dictionary at byte 0x15e for key /ColorSpace
Multiple definitions in dictionary at byte 0x16a for key /Height
Multiple definitions in dictionary at byte 0x17b for key /Filter
Multiple definitions in dictionary at byte 0x185 for key /Width
Multiple definitions in dictionary at byte 0x192 for key /Auto
Multiple definitions in dictionary at byte 0x1a5 for key /BitsPerComponent
Multiple definitions in dictionary at byte 0x183f6 for key /Length
Multiple definitions in dictionary at byte 0x18409 for key /Filter
Multiple definitions in dictionary at byte 0x1a925 for key /Length
Multiple definitions in dictionary at byte 0x1a938 for key /Filter
Multiple 

In [ ]:
# %%

diagnosis_db = Chroma.from_documents(
    texts,
    ollama_embeddings,
    persist_directory=chroma_path,
    collection_metadata={"hnsw:space": "cosine"},
)

In [ ]:
# %%

diagnosis_db = Chroma(
    embedding_function=ollama_embeddings,
    persist_directory=chroma_path,
)

# # %%
# chroma_path = "chroma_db/heart_disease"

# db_loader = CSVLoader(
#     file_path="heart_disease.csv",
#     csv_args={
#         "delimiter": ",",
#         "quotechar": '"',
#         "fieldnames": [
#             "age",
#             "sex",
#             "cp",
#             "trestbps",
#             "chol",
#             "fbs",
#             "restecg",
#             "thalach",
#             "exang",
#             "oldpeak",
#             "slope",
#             "ca",
#             "thal",
#             "target",
#         ],
#     },
# )

# loader = db_loader.load()
# heart_db = Chroma(
#     collection_name="heart_disease",
#     embedding_function=ollama_embeddings,
#     persist_directory=chroma_path,
# )

In [ ]:
# %%

question = "What is the most common metric of heach disease?"
from langchain_community.llms.openai import OpenAI
from langchain.chains.retrieval_qa.base import RetrievalQA
from langchain.retrievers.self_query.base import SelfQueryRetriever
from langchain.chains.query_constructor.base import AttributeInfo

llm = OpenAI(base_url=base_url, temperature=0)

qa_chain = RetrievalQA.from_chain_type(llm, retriever=diagnosis_db.as_retriever())
result = qa_chain({"query": question})
print(result["result"])

/Users/ric/Library/Caches/pypoetry/virtualenvs/cardio-KphtDLjZ-py3.11/lib/python3.11/site-packages/langchain_core/_api/deprecation.py:119: LangChainDeprecationWarning: The class `OpenAI` was deprecated in LangChain 0.0.10 and will be removed in 0.3.0. An updated version of the class exists in the langchain-openai package and should be used instead. To use it run `pip install -U langchain-openai` and import as `from langchain_openai import OpenAI`.
  warn_deprecated(
/Users/ric/Library/Caches/pypoetry/virtualenvs/cardio-KphtDLjZ-py3.11/lib/python3.11/site-packages/langchain_core/_api/deprecation.py:119: LangChainDeprecationWarning: The method `Chain.__call__` was deprecated in langchain 0.1.0 and will be removed in 0.3.0. Use invoke instead.
  warn_deprecated(


 The most common metric for measuring headache disease is the number of days per month that a person experiences headaches. This is often referred to as the "frequency" of headaches. Other metrics include the severity and duration of headaches, as well as the impact on daily activities such as work or school performance.


In [ ]:
# %%

question = "What is the most common metric of each heart disease?"
from langchain_community.llms.openai import OpenAI
from langchain.chains.retrieval_qa.base import RetrievalQA
from langchain.retrievers.self_query.base import SelfQueryRetriever
from langchain.chains.query_constructor.base import AttributeInfo

llm = OpenAI(base_url=base_url, temperature=0)

qa_chain = RetrievalQA.from_chain_type(llm, retriever=diagnosis_db.as_retriever())
result = qa_chain({"query": question})
print(result["result"])

 The most common metric for right ventricular function (RVF) is tricuspid annular plane systolic excursion (TAPSE). For right ventricular hypertrophy (RVH), it is relative wall thickness (RWT). For right ventricular inflow tract (RVIT), it is peak velocity of the tricuspid regurgitation jet.


In [ ]:
# %%

metadata_field_info = [
    AttributeInfo(
        name="source",
        description="The lecture the chunk is from, should be one of `docs/cs229_lectures/MachineLearning-Lecture01.pdf`, `docs/cs229_lectures/MachineLearning-Lecture02.pdf`, or `docs/cs229_lectures/MachineLearning-Lecture03.pdf`",
        type="string",
    ),
    AttributeInfo(
        name="page",
        description="The page from the lecture",
        type="integer",
    ),
]

In [ ]:
# %%

import lark

content = "anonimous data from heart diseases patients"
retriever = SelfQueryRetriever.from_llm(
    llm,
    diagnosis_db,
    content,
    metadata_field_info=metadata_field_info,
    embedding_function=ollama_embeddings,
)

docs = retriever.get_relevant_documents(
    question,
    # {"query": question, "filter": "heart_disease.csv"},
    k=100,
    # type="mmr"
)

/Users/ric/Library/Caches/pypoetry/virtualenvs/cardio-KphtDLjZ-py3.11/lib/python3.11/site-packages/langchain_core/_api/deprecation.py:119: LangChainDeprecationWarning: The method `BaseRetriever.get_relevant_documents` was deprecated in langchain-core 0.1.46 and will be removed in 0.3.0. Use invoke instead.
  warn_deprecated(


OutputParserException: Parsing text
```json
{
    "query": "heart disease",
    "filter": "eq(\"source\", \"docs/cs229_lectures/MachineLearning-Lecture01.pdf\") and eq(\"page\", 3)"
}
```
 raised following error:
Unexpected token Token('CNAME', 'and') at line 1, column 67.
Expected one of: 
	* $END
Previous tokens: [Token('RPAR', ')')]


In [ ]:
# %%

metadata_field_info = [
    AttributeInfo(
        name="source",
        description="The lecture the chunk is from, should be one of `data/oxford-handbook-of-cardiology-2nbsped-9780199643219_compress.pdf`",
        type="string",
    ),
    AttributeInfo(
        name="page",
        description="The page from the lecture",
        type="integer",
    ),
]

In [ ]:
# %%

import lark

content = "anonimous data from heart diseases patients"
retriever = SelfQueryRetriever.from_llm(
    llm,
    diagnosis_db,
    content,
    metadata_field_info=metadata_field_info,
    embedding_function=ollama_embeddings,
)

docs = retriever.get_relevant_documents(
    question,
    # {"query": question, "filter": "heart_disease.csv"},
    k=100,
    # type="mmr"
)

OutputParserException: Parsing text
```json
{
    "query": "heart disease",
    "filter": "eq(\"source\", \"data/oxford-handbook-of-cardiology-2nbsped-9780199643219_compress.pdf\") and eq(attr(\"page\"), max(attr(\"page\")))"
}
```
 raised following error:
Unexpected token Token('CNAME', 'and') at line 1, column 87.
Expected one of: 
	* $END
Previous tokens: [Token('RPAR', ')')]


In [ ]:
# %%

from langchain.retrievers.self_query.base import SelfQueryRetriever
from langchain.chains.query_constructor.base import AttributeInfo

# Define metadata for better document retrieval
metadata_field_info = [
    AttributeInfo(
        name="source",
        description="The lecture the chunk is from, should be one of `data/oxford-handbook-of-cardiology-2nbsped-9780199643219_compress.pdf`",
        type="string",
    ),
    AttributeInfo(
        name="page",
        description="The page from the lecture",
        type="integer",
    ),
]

# Set up SelfQueryRetriever for advanced querying
content = "anonymous data from heart disease patients"
retriever = SelfQueryRetriever.from_llm(
    llm,
    diagnosis_db,
    content,
    metadata_field_info=metadata_field_info,
    embedding_function=ollama_embeddings,
)

# Get relevant documents for a query
docs = retriever.get_relevant_documents(
    question,
    k=100,
)

# Display retrieved documents (for debugging)
for doc in docs:
    print(doc)

OutputParserException: Parsing text
```json
{
    "query": "heart disease",
    "filter": "eq(\"source\", \"data/oxford-handbook-of-cardiology-2nbsped-9780199643219_compress.pdf\") and eq(attr(\"page\"), max(attr(\"page\")))"
}
```
 raised following error:
Unexpected token Token('CNAME', 'and') at line 1, column 87.
Expected one of: 
	* $END
Previous tokens: [Token('RPAR', ')')]


In [ ]:
# %%

# Assuming you're setting up a LangChain-based retriever
from langchain.chains.query_constructor.base import AttributeInfo

metadata_field_info = [
    AttributeInfo(
        name="source",
        description="The lecture the chunk is from, should be one of `data/oxford-handbook-of-cardiology-2nbsped-9780199643219_compress.pdf`",
        type="string",
    ),
    AttributeInfo(
        name="page",
        description="The page from the lecture",
        type="integer",
    ),
]

# Proper query construction
content = "anonymous data from heart disease patients"
retriever = SelfQueryRetriever.from_llm(
    llm,
    diagnosis_db,
    content,
    metadata_field_info=metadata_field_info,
    embedding_function=ollama_embeddings,
)

# Example of a correct query
question = "What is the most common metric of each heart disease?"
docs = retriever.get_relevant_documents(
    question,
    k=100,
)

for doc in docs:
    print(doc)

OutputParserException: Parsing text
```json
{
    "query": "heart disease",
    "filter": "eq(\"source\", \"data/oxford-handbook-of-cardiology-2nbsped-9780199643219_compress.pdf\") and eq(attr(\"page\"), max(attr(\"page\")))"
}
```
 raised following error:
Unexpected token Token('CNAME', 'and') at line 1, column 87.
Expected one of: 
	* $END
Previous tokens: [Token('RPAR', ')')]


In [ ]:
# %%

# Assuming you're setting up a LangChain-based retriever
from langchain.chains.query_constructor.base import AttributeInfo

metadata_field_info = [
    AttributeInfo(
        name="page",
        description="The page from the lecture",
        type="integer",
    ),
]

# Proper query construction
content = "anonymous data from heart disease patients"
retriever = SelfQueryRetriever.from_llm(
    llm,
    diagnosis_db,
    content,
    metadata_field_info=metadata_field_info,
    embedding_function=ollama_embeddings,
)

# Example of a correct query
question = "What is the most common metric of each heart disease?"
docs = retriever.get_relevant_documents(
    question,
    k=100,
)

for doc in docs:
    print(doc)

page_content='Philadelphia, PA WB Saunders.     Acute coronary syndromes\nNo ST elevation ST elevation\nNSTEMI\nUnstable angina NQ-MI QwMINo\nenzyme\nrise' metadata={'page': 279, 'source': 'data/oxford-handbook-of-cardiology-2nbsped-9780199643219_compress.pdf'}
page_content='Following withdrawal due to hyperkalaemia, restart at a dose of 25 mg \nevery other day if K  +  <5.0 mmol/L.        Table 2.6   Doses for commonly used aldosterone antagonists  \n  Agent    Starting dose    Target dose  \n Spironolactone \n  Heart failure \n  Hypertension  25 mg daily \n 100 mg daily  50 mg daily a  \n 25–200 mg daily \n Eplerenone  25 mg daily  50 mg daily a  \n      Table 2.7   Recommended dose adjustments based on potassium levels  \n  Serum K  +   (mmol/L)    Action    Dose adjustment  \n <5.0  Increase dose  25 mg every other day to 25 mg daily \n 25 mg daily to 50 mg daily \n 5.0–5.4  Maintain dose  No dose adjustment \n 5.5–5.9  Decrease dose  50 mg daily to 25 mg daily \n 25 mg daily to 25

In [ ]:
# %%

metadata_field_info = [
    AttributeInfo(
        name="source",
        description="The lecture the chunk is from, should be one of `data/oxford-handbook-of-cardiology-2nbsped-9780199643219_compress.pdf`",
        type="string",
    ) or
    AttributeInfo(
        name="page",
        description="The page from the lecture",
        type="integer",
    ),
]

In [ ]:
# %%

import lark

content = "anonimous data from heart diseases patients"
retriever = SelfQueryRetriever.from_llm(
    llm,
    diagnosis_db,
    content,
    metadata_field_info=metadata_field_info,
    embedding_function=ollama_embeddings,
)

docs = retriever.get_relevant_documents(
    question,
    # {"query": question, "filter": "heart_disease.csv"},
    k=100,
    # type="mmr"
)
for doc in docs:
    print(doc)

page_content='This page intentionally left blank' metadata={'page': 387, 'source': 'data/oxford-handbook-of-cardiology-2nbsped-9780199643219_compress.pdf'}
page_content='This page intentionally left blank' metadata={'page': 409, 'source': 'data/oxford-handbook-of-cardiology-2nbsped-9780199643219_compress.pdf'}
page_content='This page intentionally left blank' metadata={'page': 467, 'source': 'data/oxford-handbook-of-cardiology-2nbsped-9780199643219_compress.pdf'}
page_content='This page intentionally left blank' metadata={'page': 491, 'source': 'data/oxford-handbook-of-cardiology-2nbsped-9780199643219_compress.pdf'}


In [ ]:
# %%

# Proper query construction
content = "anonymous data from heart disease patients"
retriever = SelfQueryRetriever.from_llm(
    llm,
    diagnosis_db,
    content,
    metadata_field_info=metadata_field_info,
    embedding_function=ollama_embeddings,
)

# Example of a correct query
question = "What is the most common metric of each heart disease?"
docs = retriever.get_relevant_documents(
    question,
    k=100,
)

for doc in docs:
    print(doc)

page_content='This page intentionally left blank' metadata={'page': 387, 'source': 'data/oxford-handbook-of-cardiology-2nbsped-9780199643219_compress.pdf'}
page_content='This page intentionally left blank' metadata={'page': 409, 'source': 'data/oxford-handbook-of-cardiology-2nbsped-9780199643219_compress.pdf'}
page_content='This page intentionally left blank' metadata={'page': 467, 'source': 'data/oxford-handbook-of-cardiology-2nbsped-9780199643219_compress.pdf'}
page_content='This page intentionally left blank' metadata={'page': 491, 'source': 'data/oxford-handbook-of-cardiology-2nbsped-9780199643219_compress.pdf'}


In [ ]:
# %%

import lark

content = "anonimous data from heart diseases patients"
retriever = SelfQueryRetriever.from_llm(
    llm,
    diagnosis_db,
    content,
    metadata_field_info=metadata_field_info,
    embedding_function=ollama_embeddings,
)

docs = retriever.get_relevant_documents(
    question,
    {"query": question, "filter": "heart_disease.csv"},
    k=100,
    type="mmr"
)
for doc in docs:
    print(doc)

TypeError: BaseRetriever.get_relevant_documents() takes 2 positional arguments but 3 were given

In [ ]:
# %%

import lark

content = "anonimous data from heart diseases patients"
retriever = SelfQueryRetriever.from_llm(
    llm,
    diagnosis_db,
    content,
    metadata_field_info=metadata_field_info,
    embedding_function=ollama_embeddings,
)

docs = retriever.get_relevant_documents(
    question,
    # {"query": question, "filter": "heart_disease.csv"},
    k=100,
    type="mmr",
)
for doc in docs:
    print(doc)

page_content='This page intentionally left blank' metadata={'page': 387, 'source': 'data/oxford-handbook-of-cardiology-2nbsped-9780199643219_compress.pdf'}
page_content='This page intentionally left blank' metadata={'page': 409, 'source': 'data/oxford-handbook-of-cardiology-2nbsped-9780199643219_compress.pdf'}
page_content='This page intentionally left blank' metadata={'page': 467, 'source': 'data/oxford-handbook-of-cardiology-2nbsped-9780199643219_compress.pdf'}
page_content='This page intentionally left blank' metadata={'page': 491, 'source': 'data/oxford-handbook-of-cardiology-2nbsped-9780199643219_compress.pdf'}


In [ ]:
# %%

import lark

content = "anonimous data from heart diseases patients"
retriever = SelfQueryRetriever.from_llm(
    llm,
    diagnosis_db,
    content,
    metadata_field_info=metadata_field_info,
    embedding_function=ollama_embeddings,
)

docs = retriever.get_relevant_documents(
    question,
    # {"query": question, "filter": "heart_disease.csv"},
    k=20,
    type="mmr",
)
for doc in docs:
    print(doc)

page_content='This page intentionally left blank' metadata={'page': 387, 'source': 'data/oxford-handbook-of-cardiology-2nbsped-9780199643219_compress.pdf'}
page_content='This page intentionally left blank' metadata={'page': 409, 'source': 'data/oxford-handbook-of-cardiology-2nbsped-9780199643219_compress.pdf'}
page_content='This page intentionally left blank' metadata={'page': 467, 'source': 'data/oxford-handbook-of-cardiology-2nbsped-9780199643219_compress.pdf'}
page_content='This page intentionally left blank' metadata={'page': 491, 'source': 'data/oxford-handbook-of-cardiology-2nbsped-9780199643219_compress.pdf'}


In [ ]:
# %%

import lark

content = "standarf cardiology diagnosis"
retriever = SelfQueryRetriever.from_llm(
    llm,
    diagnosis_db,
    content,
    metadata_field_info=metadata_field_info,
    embedding_function=ollama_embeddings,
)

docs = retriever.get_relevant_documents(
    question,
    # {"query": question, "filter": "heart_disease.csv"},
    k=20,
    type="mmr",
)
for doc in docs:
    print(doc)

page_content='This page intentionally left blank' metadata={'page': 387, 'source': 'data/oxford-handbook-of-cardiology-2nbsped-9780199643219_compress.pdf'}
page_content='This page intentionally left blank' metadata={'page': 409, 'source': 'data/oxford-handbook-of-cardiology-2nbsped-9780199643219_compress.pdf'}
page_content='This page intentionally left blank' metadata={'page': 467, 'source': 'data/oxford-handbook-of-cardiology-2nbsped-9780199643219_compress.pdf'}
page_content='This page intentionally left blank' metadata={'page': 491, 'source': 'data/oxford-handbook-of-cardiology-2nbsped-9780199643219_compress.pdf'}


In [ ]:
# %%

metadata_field_info = [
    AttributeInfo(
        name="source",
        description="The lecture the chunk is from, should be one of `data/oxford-handbook-of-cardiology-2nbsped-9780199643219_compress.pdf`",
        type="string",
    )
    # or
    and
     AttributeInfo(
        name="page",
        description="The page from the lecture",
        type="integer",
    ),
]

In [ ]:
# %%

import lark

content = "standarf cardiology diagnosis"
retriever = SelfQueryRetriever.from_llm(
    llm,
    diagnosis_db,
    content,
    metadata_field_info=metadata_field_info,
    embedding_function=ollama_embeddings,
)

docs = retriever.get_relevant_documents(
    question,
    # {"query": question, "filter": "heart_disease.csv"},
    k=20,
    type="mmr",
)
for doc in docs:
    print(doc)

page_content='OXFORD MEDICAL PUBLICATIONS    \n Oxford Handbook of \n Cardiology' metadata={'page': 1, 'source': 'data/oxford-handbook-of-cardiology-2nbsped-9780199643219_compress.pdf'}


In [ ]:
# %%

question = "What is the most common metric of each heart disease?"
from langchain_community.llms.openai import OpenAI
from langchain.chains.retrieval_qa.base import RetrievalQA
from langchain.retrievers.self_query.base import SelfQueryRetriever
from langchain.chains.query_constructor.base import AttributeInfo

llm = OpenAI(base_url=base_url, temperature=0)

qa_chain = RetrievalQA.from_chain_type(llm, retriever=diagnosis_db.as_retriever())
result = qa_chain({"query": question})
print(result["result"])

 The most common metric for right ventricular function (RVF) is tricuspid annular plane systolic excursion (TAPSE). For right ventricular hypertrophy (RVH), it is relative wall thickness (RWT). For right ventricular inflow tract (RVIT), it is peak velocity of the tricuspid regurgitation jet.


In [ ]:
# %%

import lark

content = "standarf cardiology diagnosis"
retriever = SelfQueryRetriever.from_llm(
    llm,
    diagnosis_db,
    content,
    metadata_field_info=metadata_field_info,
    embedding_function=ollama_embeddings,
)

docs = retriever.get_relevant_documents(
    question,
    # {"query": question, "filter": "heart_disease.csv"},
    k=20,
    type="mmr",
)
for doc in docs:
    print(doc)

page_content='OXFORD MEDICAL PUBLICATIONS    \n Oxford Handbook of \n Cardiology' metadata={'page': 1, 'source': 'data/oxford-handbook-of-cardiology-2nbsped-9780199643219_compress.pdf'}


In [ ]:
# %%

from langchain_community.document_loaders.pdf import PyPDFLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.embeddings.ollama import OllamaEmbeddings
from langchain_community.vectorstores.chroma import Chroma

doc_path = "data/oxford-handbook-of-cardiology-2nbsped-9780199643219_compress.pdf"
ollama_url = "http://192.168.0.100:11434"
base_url = "http://192.168.0.100:8000/v1"
chroma_path = "chroma_db/diad_man"
ollama_embeddings = OllamaEmbeddings(model="mxbai-embed-large", base_url=ollama_url)

In [ ]:
# %%

diagnosis_db = Chroma(
    embedding_function=ollama_embeddings,
    persist_directory=chroma_path,
)

# # %%
# chroma_path = "chroma_db/heart_disease"

# db_loader = CSVLoader(
#     file_path="heart_disease.csv",
#     csv_args={
#         "delimiter": ",",
#         "quotechar": '"',
#         "fieldnames": [
#             "age",
#             "sex",
#             "cp",
#             "trestbps",
#             "chol",
#             "fbs",
#             "restecg",
#             "thalach",
#             "exang",
#             "oldpeak",
#             "slope",
#             "ca",
#             "thal",
#             "target",
#         ],
#     },
# )

# loader = db_loader.load()
# heart_db = Chroma(
#     collection_name="heart_disease",
#     embedding_function=ollama_embeddings,
#     persist_directory=chroma_path,
# )

In [ ]:
# %%

question = "What is the most common metric of each heart disease?"
from langchain_community.llms.openai import OpenAI
from langchain.chains.retrieval_qa.base import RetrievalQA
from langchain.retrievers.self_query.base import SelfQueryRetriever
from langchain.chains.query_constructor.base import AttributeInfo

llm = OpenAI(base_url=base_url, temperature=0)

qa_chain = RetrievalQA.from_chain_type(llm, retriever=diagnosis_db.as_retriever())
result = qa_chain({"query": question})
print(result["result"])

 The most common metric for right ventricular dysfunction (RVF) is ejection fraction (EF). For right ventricular hypertrophy (RVH), it is wall thickness. For right ventricular inflow tract (RVIT), it is peak velocity of the tricuspid regurgitation jet.


In [ ]:
# %%

metadata_field_info = [
    AttributeInfo(
        name="source",
        description="The lecture the chunk is from, should be one of `data/oxford-handbook-of-cardiology-2nbsped-9780199643219_compress.pdf`",
        type="string",
    )
    # or
    and AttributeInfo(
        name="page",
        description="The page from the lecture",
        type="integer",
    ),
]

In [ ]:
# %%

import lark

content = "standarf cardiology diagnosis"
retriever = SelfQueryRetriever.from_llm(
    llm,
    diagnosis_db,
    content,
    metadata_field_info=metadata_field_info,
    embedding_function=ollama_embeddings,
)

docs = retriever.get_relevant_documents(
    question,
    # {"query": question, "filter": "heart_disease.csv"},
    k=100,
    type="mmr",
)
for doc in docs:
    print(doc)

OutputParserException: Parsing text
```json
{
    "query": "heart disease",
    "filter": "eq(\"page\", 1) or eq(\"page\", 2) or eq(\"page\", 3)"
}
```
 raised following error:
Unexpected token Token('CNAME', 'or') at line 1, column 15.
Expected one of: 
	* $END
Previous tokens: [Token('RPAR', ')')]


In [ ]:
# %%

metadata_field_info = [
    AttributeInfo(
        name="source",
        description="The lecture the chunk is from, should be one of `data/oxford-handbook-of-cardiology-2nbsped-9780199643219_compress.pdf`",
        type="string",
    )
    or
    # and
    AttributeInfo(
        name="page",
        description="The page from the lecture",
        type="integer",
    ),
]

In [ ]:
# %%

import lark

content = "standarf cardiology diagnosis"
retriever = SelfQueryRetriever.from_llm(
    llm,
    diagnosis_db,
    content,
    metadata_field_info=metadata_field_info,
    embedding_function=ollama_embeddings,
)

docs = retriever.get_relevant_documents(
    question,
    # {"query": question, "filter": "heart_disease.csv"},
    k=100,
    type="mmr",
)
for doc in docs:
    print(doc)

OutputParserException: Parsing text
```json
{
    "query": "heart disease",
    "filter": "eq(\"source\", \"data/oxford-handbook-of-cardiology-2nbsped-9780199643219_compress.pdf\") and (or(eq(\"metric\", \"chest pain\"), eq(\"metric\", \"dyspnea\"), eq(\"metric\", \"fatigue\"), eq(\"metric\", \"palpitations\")))"
}
```
 raised following error:
Unexpected token Token('CNAME', 'and') at line 1, column 87.
Expected one of: 
	* $END
Previous tokens: [Token('RPAR', ')')]


In [ ]:
# %%

import lark

content = "standarf cardiology diagnosis"
retriever = SelfQueryRetriever.from_llm(
    llm,
    diagnosis_db,
    content,
    metadata_field_info=metadata_field_info,
    embedding_function=ollama_embeddings,
)

docs = retriever.get_relevant_documents(
    question,
    # {"query": question, "filter": "heart_disease.csv"},
    k=100,
    type="mmr",
)
for doc in docs:
    print(doc)

OutputParserException: Parsing text
```json
{
    "query": "heart disease",
    "filter": "eq(\"source\", \"data/oxford-handbook-of-cardiology-2nbsped-9780199643219_compress.pdf\") and (or(eq(\"metric\", \"chest pain\"), eq(\"metric\", \"dyspnea\"), eq(\"metric\", \"fatigue\"), eq(\"metric\", \"palpitations\")))"
}
```
 raised following error:
Unexpected token Token('CNAME', 'and') at line 1, column 87.
Expected one of: 
	* $END
Previous tokens: [Token('RPAR', ')')]


In [ ]:
# %%

# Assuming you're setting up a LangChain-based retriever
from langchain.chains.query_constructor.base import AttributeInfo

metadata_field_info = [
    AttributeInfo(
        name="page",
        description="The page from the lecture",
        type="integer",
    ),
]

In [ ]:
# %%

# Assuming you're setting up a LangChain-based retriever
from langchain.chains.query_constructor.base import AttributeInfo

metadata_field_info = [
    AttributeInfo(
        name="page",
        description="The page from the lecture",
        type="integer",
    ),
]

In [ ]:
# %%

# Proper query construction
content = "anonymous data from heart disease patients"
retriever = SelfQueryRetriever.from_llm(
    llm,
    diagnosis_db,
    content,
    metadata_field_info=metadata_field_info,
    embedding_function=ollama_embeddings,
)

# Example of a correct query
question = "What is the most common metric of each heart disease?"
docs = retriever.get_relevant_documents(
    question,
    k=100,
)

for doc in docs:
    print(doc)

page_content='Philadelphia, PA WB Saunders.     Acute coronary syndromes\nNo ST elevation ST elevation\nNSTEMI\nUnstable angina NQ-MI QwMINo\nenzyme\nrise' metadata={'page': 279, 'source': 'data/oxford-handbook-of-cardiology-2nbsped-9780199643219_compress.pdf'}
page_content='Following withdrawal due to hyperkalaemia, restart at a dose of 25 mg \nevery other day if K  +  <5.0 mmol/L.        Table 2.6   Doses for commonly used aldosterone antagonists  \n  Agent    Starting dose    Target dose  \n Spironolactone \n  Heart failure \n  Hypertension  25 mg daily \n 100 mg daily  50 mg daily a  \n 25–200 mg daily \n Eplerenone  25 mg daily  50 mg daily a  \n      Table 2.7   Recommended dose adjustments based on potassium levels  \n  Serum K  +   (mmol/L)    Action    Dose adjustment  \n <5.0  Increase dose  25 mg every other day to 25 mg daily \n 25 mg daily to 50 mg daily \n 5.0–5.4  Maintain dose  No dose adjustment \n 5.5–5.9  Decrease dose  50 mg daily to 25 mg daily \n 25 mg daily to 25

In [ ]:
# %%

# Proper query construction
content = "anonymous data from heart disease patients"
retriever = SelfQueryRetriever.from_llm(
    llm,
    diagnosis_db,
    content,
    metadata_field_info=metadata_field_info,
    embedding_function=ollama_embeddings,
)

# Example of a correct query
question = "What is the most common metric of each heart disease?"
docs = retriever.get_relevant_documents(
    question,
    k=100,
)

for doc in docs:
    print(doc)

page_content='Philadelphia, PA WB Saunders.     Acute coronary syndromes\nNo ST elevation ST elevation\nNSTEMI\nUnstable angina NQ-MI QwMINo\nenzyme\nrise' metadata={'page': 279, 'source': 'data/oxford-handbook-of-cardiology-2nbsped-9780199643219_compress.pdf'}
page_content='Following withdrawal due to hyperkalaemia, restart at a dose of 25 mg \nevery other day if K  +  <5.0 mmol/L.        Table 2.6   Doses for commonly used aldosterone antagonists  \n  Agent    Starting dose    Target dose  \n Spironolactone \n  Heart failure \n  Hypertension  25 mg daily \n 100 mg daily  50 mg daily a  \n 25–200 mg daily \n Eplerenone  25 mg daily  50 mg daily a  \n      Table 2.7   Recommended dose adjustments based on potassium levels  \n  Serum K  +   (mmol/L)    Action    Dose adjustment  \n <5.0  Increase dose  25 mg every other day to 25 mg daily \n 25 mg daily to 50 mg daily \n 5.0–5.4  Maintain dose  No dose adjustment \n 5.5–5.9  Decrease dose  50 mg daily to 25 mg daily \n 25 mg daily to 25

In [ ]:
# %%

from langchain_community.document_loaders.pdf import PyPDFLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.embeddings.ollama import OllamaEmbeddings
from langchain_community.vectorstores.chroma import Chroma

doc_path = "data/oxford-handbook-of-cardiology-2nbsped-9780199643219_compress.pdf"
ollama_url = "http://192.168.0.100:11434"
base_url = "http://192.168.0.100:8000/v1"
chroma_path = "chroma_db/diad_man"
ollama_embeddings = OllamaEmbeddings(model="mxbai-embed-large", base_url=ollama_url)

In [ ]:
# %%

diagnosis_db = Chroma(
    embedding_function=ollama_embeddings,
    persist_directory=chroma_path,
)

# # %%
# chroma_path = "chroma_db/heart_disease"

# db_loader = CSVLoader(
#     file_path="heart_disease.csv",
#     csv_args={
#         "delimiter": ",",
#         "quotechar": '"',
#         "fieldnames": [
#             "age",
#             "sex",
#             "cp",
#             "trestbps",
#             "chol",
#             "fbs",
#             "restecg",
#             "thalach",
#             "exang",
#             "oldpeak",
#             "slope",
#             "ca",
#             "thal",
#             "target",
#         ],
#     },
# )

# loader = db_loader.load()
# heart_db = Chroma(
#     collection_name="heart_disease",
#     embedding_function=ollama_embeddings,
#     persist_directory=chroma_path,
# )

In [ ]:
# %%

question = "What is the most common metric of each heart disease?"
from langchain_community.llms.openai import OpenAI
from langchain.chains.retrieval_qa.base import RetrievalQA
from langchain.retrievers.self_query.base import SelfQueryRetriever
from langchain.chains.query_constructor.base import AttributeInfo

llm = OpenAI(base_url=base_url, temperature=0)

qa_chain = RetrievalQA.from_chain_type(llm, retriever=diagnosis_db.as_retriever())
result = qa_chain({"query": question})
print(result["result"])

 The most common metric for right ventricular dysfunction (RVF) is ejection fraction (EF). For right ventricular hypertrophy (RVH), it is wall thickness. For right ventricular inflow tract (RVIT), it is peak velocity of the tricuspid regurgitation jet.


In [ ]:
# %%

metadata_field_info = [
    AttributeInfo(
        name="source",
        description="The lecture the chunk is from, should be one of `data/oxford-handbook-of-cardiology-2nbsped-9780199643219_compress.pdf`",
        type="string",
    )
    or
    # and
    AttributeInfo(
        name="page",
        description="The page from the lecture",
        type="integer",
    ),
]

In [ ]:
# %%

import lark

content = "standarf cardiology diagnosis"
retriever = SelfQueryRetriever.from_llm(
    llm,
    diagnosis_db,
    content,
    metadata_field_info=metadata_field_info,
    embedding_function=ollama_embeddings,
)

docs = retriever.get_relevant_documents(
    question,
    # {"query": question, "filter": "heart_disease.csv"},
    k=100,
    type="mmr",
)
for doc in docs:
    print(doc)

OutputParserException: Parsing text
```json
{
    "query": "heart disease",
    "filter": "eq(\"source\", \"data/oxford-handbook-of-cardiology-2nbsped-9780199643219_compress.pdf\") and (or(eq(\"metric\", \"chest pain\"), eq(\"metric\", \"dyspnea\"), eq(\"metric\", \"fatigue\"), eq(\"metric\", \"palpitations\")))"
}
```
 raised following error:
Unexpected token Token('CNAME', 'and') at line 1, column 87.
Expected one of: 
	* $END
Previous tokens: [Token('RPAR', ')')]


In [ ]:
# %%

metadata_field_info = [
    AttributeInfo(
        name="source",
        description="The lecture the chunk is from, should be one of `data/oxford-handbook-of-cardiology-2nbsped-9780199643219_compress.pdf`",
        type="string",
    )
    # or
    and AttributeInfo(
        name="page",
        description="The page from the lecture",
        type="integer",
    ),
]

In [ ]:
# %%

import lark

content = "standarf cardiology diagnosis"
retriever = SelfQueryRetriever.from_llm(
    llm,
    diagnosis_db,
    content,
    metadata_field_info=metadata_field_info,
    embedding_function=ollama_embeddings,
)

docs = retriever.get_relevant_documents(
    question,
    # {"query": question, "filter": "heart_disease.csv"},
    k=100,
    type="mmr",
)
for doc in docs:
    print(doc)

OutputParserException: Parsing text
```json
{
    "query": "heart disease",
    "filter": "eq(\"page\", 1) or eq(\"page\", 2) or eq(\"page\", 3)"
}
```
 raised following error:
Unexpected token Token('CNAME', 'or') at line 1, column 15.
Expected one of: 
	* $END
Previous tokens: [Token('RPAR', ')')]


In [ ]:
# %%

metadata_field_info = [
    AttributeInfo(
        name="page",
        description="The page from the lecture",
        type="integer",
    ),
]

In [ ]:
# %%

import lark

content = "standarf cardiology diagnosis"
retriever = SelfQueryRetriever.from_llm(
    llm,
    diagnosis_db,
    content,
    metadata_field_info=metadata_field_info,
    embedding_function=ollama_embeddings,
)

docs = retriever.get_relevant_documents(
    question,
    # {"query": question, "filter": "heart_disease.csv"},
    k=100,
    type="mmr",
)
for doc in docs:
    print(doc)

OutputParserException: Parsing text
```json
{
    "query": "heart disease",
    "filter": "eq(\"page\", 1) or eq(\"page\", 2) or eq(\"page\", 3)"
}
```
 raised following error:
Unexpected token Token('CNAME', 'or') at line 1, column 15.
Expected one of: 
	* $END
Previous tokens: [Token('RPAR', ')')]


In [ ]:
# %%

metadata_field_info = [
    AttributeInfo(
        name="source",
        description="The lecture the chunk is from, should be one of `data/oxford-handbook-of-cardiology-2nbsped-9780199643219_compress.pdf`",
        type="string",
    ),
]

In [ ]:
# %%

import lark

content = "standarf cardiology diagnosis"
retriever = SelfQueryRetriever.from_llm(
    llm,
    diagnosis_db,
    content,
    metadata_field_info=metadata_field_info,
    embedding_function=ollama_embeddings,
)

docs = retriever.get_relevant_documents(
    question,
    # {"query": question, "filter": "heart_disease.csv"},
    k=100,
    type="mmr",
)
for doc in docs:
    print(doc)

OutputParserException: Parsing text
```json
{
    "query": "heart disease",
    "filter": "eq(\"source\", \"data/oxford-handbook-of-cardiology-2nbsped-9780199643219_compress.pdf\") and (or(eq(\"metric\", \"chest pain\"), eq(\"metric\", \"dyspnea\"), eq(\"metric\", \"fatigue\"), eq(\"metric\", \"palpitations\")))"
}
```
 raised following error:
Unexpected token Token('CNAME', 'and') at line 1, column 87.
Expected one of: 
	* $END
Previous tokens: [Token('RPAR', ')')]


In [ ]:
# %%

from langchain_community.document_loaders.pdf import PyPDFLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.embeddings.ollama import OllamaEmbeddings
from langchain_community.vectorstores.chroma import Chroma

doc_path = "data/oxford-handbook-of-cardiology-2nbsped-9780199643219_compress.pdf"
ollama_url = "http://192.168.0.100:11434"
base_url = "http://192.168.0.100:8000/v1"
chroma_path = "chroma_db/diad_man"
ollama_embeddings = OllamaEmbeddings(model="mxbai-embed-large", base_url=ollama_url)

In [ ]:
# %%

diagnosis_db = Chroma(
    embedding_function=ollama_embeddings,
    persist_directory=chroma_path,
)

# # %%
# chroma_path = "chroma_db/heart_disease"

# db_loader = CSVLoader(
#     file_path="heart_disease.csv",
#     csv_args={
#         "delimiter": ",",
#         "quotechar": '"',
#         "fieldnames": [
#             "age",
#             "sex",
#             "cp",
#             "trestbps",
#             "chol",
#             "fbs",
#             "restecg",
#             "thalach",
#             "exang",
#             "oldpeak",
#             "slope",
#             "ca",
#             "thal",
#             "target",
#         ],
#     },
# )

# loader = db_loader.load()
# heart_db = Chroma(
#     collection_name="heart_disease",
#     embedding_function=ollama_embeddings,
#     persist_directory=chroma_path,
# )

In [ ]:
# %%

question = "What is the most common metric of each heart disease?"
from langchain_community.llms.openai import OpenAI
from langchain.chains.retrieval_qa.base import RetrievalQA
from langchain.retrievers.self_query.base import SelfQueryRetriever
from langchain.chains.query_constructor.base import AttributeInfo

llm = OpenAI(base_url=base_url, temperature=0)

qa_chain = RetrievalQA.from_chain_type(llm, retriever=diagnosis_db.as_retriever())
result = qa_chain({"query": question})
print(result["result"])

 The most common metric for right ventricular dysfunction (RVF) is ejection fraction (EF). For right ventricular hypertrophy (RVH), it is wall thickness. For right ventricular inflow tract (RVIT), it is peak velocity of the tricuspid regurgitation jet.


In [ ]:
# %%

metadata_field_info = [
    AttributeInfo(
        name="source",
        description="The lecture the chunk is from, should be one of `data/oxford-handbook-of-cardiology-2nbsped-9780199643219_compress.pdf`",
        type="string",
    )
    or
    # and
    AttributeInfo(
        name="page",
        description="The page from the lecture",
        type="integer",
    ),
]

In [ ]:
# %%

import lark

content = "standard cardiology diagnosis"
retriever = SelfQueryRetriever.from_llm(
    llm,
    diagnosis_db,
    content,
    metadata_field_info=metadata_field_info,
    embedding_function=ollama_embeddings,
)

docs = retriever.get_relevant_documents(
    question,
    # {"query": question, "filter": "heart_disease.csv"},
    k=100,
    type="mmr",
)
for doc in docs:
    print(doc)

OutputParserException: Parsing text
```json
{
    "query": "heart disease",
    "filter": "eq(\"source\", \"data/oxford-handbook-of-cardiology-2nbsped-9780199643219_compress.pdf\") and eq(attr, \"metric\")"
}
```
 raised following error:
Unexpected token Token('CNAME', 'and') at line 1, column 87.
Expected one of: 
	* $END
Previous tokens: [Token('RPAR', ')')]


In [ ]:
# %%

metadata_field_info = [
    # AttributeInfo(
    #     name="source",
    #     description="The lecture the chunk is from, should be one of `data/oxford-handbook-of-cardiology-2nbsped-9780199643219_compress.pdf`",
    #     type="string",
    # )
    # or
    # and
    AttributeInfo(
        name="page",
        description="The page from the lecture",
        type="integer",
    ),
]

In [ ]:
# %%

import lark

content = "standard cardiology diagnosis"
retriever = SelfQueryRetriever.from_llm(
    llm,
    diagnosis_db,
    content,
    metadata_field_info=metadata_field_info,
    embedding_function=ollama_embeddings,
)

docs = retriever.get_relevant_documents(
    question,
    # {"query": question, "filter": "heart_disease.csv"},
    k=100,
    type="mmr",
)
for doc in docs:
    print(doc)

OutputParserException: Parsing text
```json
{
    "query": "heart disease",
    "filter": "eq(\"page\", 1) or eq(\"page\", 2) or eq(\"page\", 3)"
}
```
 raised following error:
Unexpected token Token('CNAME', 'or') at line 1, column 15.
Expected one of: 
	* $END
Previous tokens: [Token('RPAR', ')')]


In [ ]:
# %%

metadata_field_info = [
    # AttributeInfo(
    #     name="source",
    #     description="The lecture the chunk is from, should be one of `data/oxford-handbook-of-cardiology-2nbsped-9780199643219_compress.pdf`",
    #     type="string",
    # )
    # or
    # and
    AttributeInfo(
        name="page",
        description="The page from the lecture",
        type="integer",
    ),
]

In [ ]:
# %%

import lark

content = "standard cardiology diagnosis"
retriever = SelfQueryRetriever.from_llm(
    llm,
    diagnosis_db,
    content,
    metadata_field_info=metadata_field_info,
    embedding_function=ollama_embeddings,
)

docs = retriever.get_relevant_documents(
    question,
    # {"query": question, "filter": "heart_disease.csv"},
    k=100,
    type="mmr",
)
for doc in docs:
    print(doc)

OutputParserException: Parsing text
```json
{
    "query": "heart disease",
    "filter": "eq(\"page\", 1) or eq(\"page\", 2) or eq(\"page\", 3)"
}
```
 raised following error:
Unexpected token Token('CNAME', 'or') at line 1, column 15.
Expected one of: 
	* $END
Previous tokens: [Token('RPAR', ')')]


In [ ]:
# %%

metadata_field_info = [
    # AttributeInfo(
    #     name="source",
    #     description="The lecture the chunk is from, should be one of `data/oxford-handbook-of-cardiology-2nbsped-9780199643219_compress.pdf`",
    #     type="string",
    # )
    # or
    # and
    AttributeInfo(
        name="page",
        description="The page from the PDF",
        type="int",
    ),
]

In [ ]:
# %%

import lark

content = "standard cardiology diagnosis"
retriever = SelfQueryRetriever.from_llm(
    llm,
    diagnosis_db,
    content,
    metadata_field_info=metadata_field_info,
    embedding_function=ollama_embeddings,
)

docs = retriever.get_relevant_documents(
    question,
    # {"query": question, "filter": "heart_disease.csv"},
    k=100,
    type="mmr",
)
for doc in docs:
    print(doc)

OutputParserException: Parsing text
```json
{
    "query": "heart disease",
    "filter": "eq(\"page\", 1) or eq(\"page\", 2) or eq(\"page\", 3)"
}
```
 raised following error:
Unexpected token Token('CNAME', 'or') at line 1, column 15.
Expected one of: 
	* $END
Previous tokens: [Token('RPAR', ')')]


In [ ]:
# %%

metadata_field_info = [
    # AttributeInfo(
    #     name="source",
    #     description="The lecture the chunk is from, should be one of `data/oxford-handbook-of-cardiology-2nbsped-9780199643219_compress.pdf`",
    #     type="string",
    # )
    # or
    # and
    AttributeInfo(
        name="page",
        type="int",
    ),
]

ValidationError: 1 validation error for AttributeInfo
description
  field required (type=value_error.missing)

In [ ]:
# %%

metadata_field_info = [
    # AttributeInfo(
    #     name="source",
    #     description="The lecture the chunk is from, should be one of `data/oxford-handbook-of-cardiology-2nbsped-9780199643219_compress.pdf`",
    #     type="string",
    # )
    # or
    # and
    AttributeInfo(
        name="page",
        type="integer",
    ),
]

ValidationError: 1 validation error for AttributeInfo
description
  field required (type=value_error.missing)

In [ ]:
# %%

import lark

content = "standard cardiology diagnosis"
retriever = SelfQueryRetriever.from_llm(
    llm,
    diagnosis_db,
    content,
    metadata_field_info=metadata_field_info,
    embedding_function=ollama_embeddings,
)

docs = retriever.get_relevant_documents(
    # question,
    # {"query": question, "filter": "heart_disease.csv"},
    {"query": question, "filter": ""},
    k=100,
    type="mmr",
)
for doc in docs:
    print(doc)

page_content='Philadelphia, PA WB Saunders.     Acute coronary syndromes\nNo ST elevation ST elevation\nNSTEMI\nUnstable angina NQ-MI QwMINo\nenzyme\nrise' metadata={'page': 279, 'source': 'data/oxford-handbook-of-cardiology-2nbsped-9780199643219_compress.pdf'}
page_content='Following withdrawal due to hyperkalaemia, restart at a dose of 25 mg \nevery other day if K  +  <5.0 mmol/L.        Table 2.6   Doses for commonly used aldosterone antagonists  \n  Agent    Starting dose    Target dose  \n Spironolactone \n  Heart failure \n  Hypertension  25 mg daily \n 100 mg daily  50 mg daily a  \n 25–200 mg daily \n Eplerenone  25 mg daily  50 mg daily a  \n      Table 2.7   Recommended dose adjustments based on potassium levels  \n  Serum K  +   (mmol/L)    Action    Dose adjustment  \n <5.0  Increase dose  25 mg every other day to 25 mg daily \n 25 mg daily to 50 mg daily \n 5.0–5.4  Maintain dose  No dose adjustment \n 5.5–5.9  Decrease dose  50 mg daily to 25 mg daily \n 25 mg daily to 25

In [ ]:
# %%

metadata_field_info = [
    AttributeInfo(
        name="source",
        description="The lecture the chunk is from, should be one of `data/oxford-handbook-of-cardiology-2nbsped-9780199643219_compress.pdf`",
        type="string",
    )
    or
    # and
    AttributeInfo(
        name="page",
        description="The page from the PDF",
        type="int",
    ),
]

In [ ]:
# %%

import lark

content = "standard cardiology diagnosis"
retriever = SelfQueryRetriever.from_llm(
    llm,
    diagnosis_db,
    content,
    metadata_field_info=metadata_field_info,
    embedding_function=ollama_embeddings,
)

docs = retriever.get_relevant_documents(
    # question,
    # {"query": question, "filter": "heart_disease.csv"},
    {"query": question, "filter": ""},
    k=100,
    type="mmr",
)
for doc in docs:
    print(doc)

page_content='Philadelphia, PA WB Saunders.     Acute coronary syndromes\nNo ST elevation ST elevation\nNSTEMI\nUnstable angina NQ-MI QwMINo\nenzyme\nrise' metadata={'page': 279, 'source': 'data/oxford-handbook-of-cardiology-2nbsped-9780199643219_compress.pdf'}
page_content='Following withdrawal due to hyperkalaemia, restart at a dose of 25 mg \nevery other day if K  +  <5.0 mmol/L.        Table 2.6   Doses for commonly used aldosterone antagonists  \n  Agent    Starting dose    Target dose  \n Spironolactone \n  Heart failure \n  Hypertension  25 mg daily \n 100 mg daily  50 mg daily a  \n 25–200 mg daily \n Eplerenone  25 mg daily  50 mg daily a  \n      Table 2.7   Recommended dose adjustments based on potassium levels  \n  Serum K  +   (mmol/L)    Action    Dose adjustment  \n <5.0  Increase dose  25 mg every other day to 25 mg daily \n 25 mg daily to 50 mg daily \n 5.0–5.4  Maintain dose  No dose adjustment \n 5.5–5.9  Decrease dose  50 mg daily to 25 mg daily \n 25 mg daily to 25

In [ ]:
# %%

for doc in docs:
    print(doc.page_content)

Philadelphia, PA WB Saunders.     Acute coronary syndromes
No ST elevation ST elevation
NSTEMI
Unstable angina NQ-MI QwMINo
enzyme
rise
Following withdrawal due to hyperkalaemia, restart at a dose of 25 mg 
every other day if K  +  <5.0 mmol/L.        Table 2.6   Doses for commonly used aldosterone antagonists  
  Agent    Starting dose    Target dose  
 Spironolactone 
  Heart failure 
  Hypertension  25 mg daily 
 100 mg daily  50 mg daily a  
 25–200 mg daily 
 Eplerenone  25 mg daily  50 mg daily a  
      Table 2.7   Recommended dose adjustments based on potassium levels  
  Serum K  +   (mmol/L)    Action    Dose adjustment  
 <5.0  Increase dose  25 mg every other day to 25 mg daily 
 25 mg daily to 50 mg daily 
 5.0–5.4  Maintain dose  No dose adjustment 
 5.5–5.9  Decrease dose  50 mg daily to 25 mg daily 
 25 mg daily to 25 mg every other day 
 25 mg every other day to withhold drug 
  t 6.0  Withhold drug
Agent    Starting dose    Target dose  
 Candesartan: 
  Hypertension 

In [ ]:
# %%

for doc in docs:
    print(doc.page_content)
    print(doc.metadata)

Philadelphia, PA WB Saunders.     Acute coronary syndromes
No ST elevation ST elevation
NSTEMI
Unstable angina NQ-MI QwMINo
enzyme
rise
{'page': 279, 'source': 'data/oxford-handbook-of-cardiology-2nbsped-9780199643219_compress.pdf'}
Following withdrawal due to hyperkalaemia, restart at a dose of 25 mg 
every other day if K  +  <5.0 mmol/L.        Table 2.6   Doses for commonly used aldosterone antagonists  
  Agent    Starting dose    Target dose  
 Spironolactone 
  Heart failure 
  Hypertension  25 mg daily 
 100 mg daily  50 mg daily a  
 25–200 mg daily 
 Eplerenone  25 mg daily  50 mg daily a  
      Table 2.7   Recommended dose adjustments based on potassium levels  
  Serum K  +   (mmol/L)    Action    Dose adjustment  
 <5.0  Increase dose  25 mg every other day to 25 mg daily 
 25 mg daily to 50 mg daily 
 5.0–5.4  Maintain dose  No dose adjustment 
 5.5–5.9  Decrease dose  50 mg daily to 25 mg daily 
 25 mg daily to 25 mg every other day 
 25 mg every other day to withhold dr

In [ ]:
# %%

# Proper query construction
content = "anonymous data from heart disease patients"
retriever = SelfQueryRetriever.from_llm(
    llm,
    diagnosis_db,
    content,
    metadata_field_info=metadata_field_info,
    embedding_function=ollama_embeddings,
)

# Example of a correct query
question = "What is the most common metric of each heart disease?"
docs_ss = retriever.get_relevant_documents(
    question,
    k=100,
)

for doc in docs_ss:
    print(doc.page_content)
    print(doc.metadata)

This page intentionally left blank
{'page': 387, 'source': 'data/oxford-handbook-of-cardiology-2nbsped-9780199643219_compress.pdf'}
This page intentionally left blank
{'page': 409, 'source': 'data/oxford-handbook-of-cardiology-2nbsped-9780199643219_compress.pdf'}
This page intentionally left blank
{'page': 467, 'source': 'data/oxford-handbook-of-cardiology-2nbsped-9780199643219_compress.pdf'}
This page intentionally left blank
{'page': 491, 'source': 'data/oxford-handbook-of-cardiology-2nbsped-9780199643219_compress.pdf'}


In [ ]:
# %%

# Proper query construction
content = "anonymous data from heart disease patients"
retriever = SelfQueryRetriever.from_llm(
    llm,
    diagnosis_db,
    content,
    metadata_field_info=metadata_field_info,
    embedding_function=ollama_embeddings,
)

# Example of a correct query
question = "What is the most common metric of each heart disease?"
docs_ss = retriever.get_relevant_documents(
    {"query": question, "filter": ""},
    k=100,
)

for doc in docs_ss:
    print(doc.page_content)
    print(doc.metadata)

Philadelphia, PA WB Saunders.     Acute coronary syndromes
No ST elevation ST elevation
NSTEMI
Unstable angina NQ-MI QwMINo
enzyme
rise
{'page': 279, 'source': 'data/oxford-handbook-of-cardiology-2nbsped-9780199643219_compress.pdf'}
Following withdrawal due to hyperkalaemia, restart at a dose of 25 mg 
every other day if K  +  <5.0 mmol/L.        Table 2.6   Doses for commonly used aldosterone antagonists  
  Agent    Starting dose    Target dose  
 Spironolactone 
  Heart failure 
  Hypertension  25 mg daily 
 100 mg daily  50 mg daily a  
 25–200 mg daily 
 Eplerenone  25 mg daily  50 mg daily a  
      Table 2.7   Recommended dose adjustments based on potassium levels  
  Serum K  +   (mmol/L)    Action    Dose adjustment  
 <5.0  Increase dose  25 mg every other day to 25 mg daily 
 25 mg daily to 50 mg daily 
 5.0–5.4  Maintain dose  No dose adjustment 
 5.5–5.9  Decrease dose  50 mg daily to 25 mg daily 
 25 mg daily to 25 mg every other day 
 25 mg every other day to withhold dr

In [ ]:
# %%

for doc in docs_mmr:
    print(doc.page_content)
    print(doc.metadata)
print(len(docs_mmr))

NameError: name 'docs_mmr' is not defined

In [ ]:
# %%

import lark

content = "standard cardiology diagnosis"
retriever = SelfQueryRetriever.from_llm(
    llm,
    diagnosis_db,
    content,
    metadata_field_info=metadata_field_info,
    embedding_function=ollama_embeddings,
)

docs_mmr = retriever.get_relevant_documents(
    # question,
    # {"query": question, "filter": "heart_disease.csv"},
    {"query": question, "filter": ""},
    k=100,
    type="mmr",
)

for doc in docs_mmr:
    print(doc)

page_content='Philadelphia, PA WB Saunders.     Acute coronary syndromes\nNo ST elevation ST elevation\nNSTEMI\nUnstable angina NQ-MI QwMINo\nenzyme\nrise' metadata={'page': 279, 'source': 'data/oxford-handbook-of-cardiology-2nbsped-9780199643219_compress.pdf'}
page_content='Following withdrawal due to hyperkalaemia, restart at a dose of 25 mg \nevery other day if K  +  <5.0 mmol/L.        Table 2.6   Doses for commonly used aldosterone antagonists  \n  Agent    Starting dose    Target dose  \n Spironolactone \n  Heart failure \n  Hypertension  25 mg daily \n 100 mg daily  50 mg daily a  \n 25–200 mg daily \n Eplerenone  25 mg daily  50 mg daily a  \n      Table 2.7   Recommended dose adjustments based on potassium levels  \n  Serum K  +   (mmol/L)    Action    Dose adjustment  \n <5.0  Increase dose  25 mg every other day to 25 mg daily \n 25 mg daily to 50 mg daily \n 5.0–5.4  Maintain dose  No dose adjustment \n 5.5–5.9  Decrease dose  50 mg daily to 25 mg daily \n 25 mg daily to 25

In [ ]:
# %%

for doc in docs_mmr:
    print(doc.page_content)
    print(doc.metadata)
print(len(docs_mmr))

Philadelphia, PA WB Saunders.     Acute coronary syndromes
No ST elevation ST elevation
NSTEMI
Unstable angina NQ-MI QwMINo
enzyme
rise
{'page': 279, 'source': 'data/oxford-handbook-of-cardiology-2nbsped-9780199643219_compress.pdf'}
Following withdrawal due to hyperkalaemia, restart at a dose of 25 mg 
every other day if K  +  <5.0 mmol/L.        Table 2.6   Doses for commonly used aldosterone antagonists  
  Agent    Starting dose    Target dose  
 Spironolactone 
  Heart failure 
  Hypertension  25 mg daily 
 100 mg daily  50 mg daily a  
 25–200 mg daily 
 Eplerenone  25 mg daily  50 mg daily a  
      Table 2.7   Recommended dose adjustments based on potassium levels  
  Serum K  +   (mmol/L)    Action    Dose adjustment  
 <5.0  Increase dose  25 mg every other day to 25 mg daily 
 25 mg daily to 50 mg daily 
 5.0–5.4  Maintain dose  No dose adjustment 
 5.5–5.9  Decrease dose  50 mg daily to 25 mg daily 
 25 mg daily to 25 mg every other day 
 25 mg every other day to withhold dr

In [ ]:
# %%

# Assuming you're setting up a LangChain-based retriever
from langchain.chains.query_constructor.base import AttributeInfo

metadata_field_info = [
    AttributeInfo(
        name="page",
        description="The page from the lecture",
        type="integer",
    ),
]

In [ ]:
# %%

# Proper query construction
content = "anonymous data from heart disease patients"
retriever = SelfQueryRetriever.from_llm(
    llm,
    diagnosis_db,
    content,
    metadata_field_info=metadata_field_info,
    embedding_function=ollama_embeddings,
)

# Example of a correct query
question = "What is the most common metric of each heart disease?"
docs_ss = retriever.get_relevant_documents(
    {"query": question, "filter": ""},
    k=100,
)

for doc in docs_ss:
    print(doc.page_content)
    print(doc.metadata)
print(len(docs_ss))

Philadelphia, PA WB Saunders.     Acute coronary syndromes
No ST elevation ST elevation
NSTEMI
Unstable angina NQ-MI QwMINo
enzyme
rise
{'page': 279, 'source': 'data/oxford-handbook-of-cardiology-2nbsped-9780199643219_compress.pdf'}
Following withdrawal due to hyperkalaemia, restart at a dose of 25 mg 
every other day if K  +  <5.0 mmol/L.        Table 2.6   Doses for commonly used aldosterone antagonists  
  Agent    Starting dose    Target dose  
 Spironolactone 
  Heart failure 
  Hypertension  25 mg daily 
 100 mg daily  50 mg daily a  
 25–200 mg daily 
 Eplerenone  25 mg daily  50 mg daily a  
      Table 2.7   Recommended dose adjustments based on potassium levels  
  Serum K  +   (mmol/L)    Action    Dose adjustment  
 <5.0  Increase dose  25 mg every other day to 25 mg daily 
 25 mg daily to 50 mg daily 
 5.0–5.4  Maintain dose  No dose adjustment 
 5.5–5.9  Decrease dose  50 mg daily to 25 mg daily 
 25 mg daily to 25 mg every other day 
 25 mg every other day to withhold dr

In [ ]:
# %%

# Proper query construction
content = "anonymous data from heart disease patients"
retriever = SelfQueryRetriever.from_llm(
    llm,
    diagnosis_db,
    content,
    metadata_field_info=metadata_field_info,
    embedding_function=ollama_embeddings,
)

# Example of a correct query
question = "What is the most common metric of each heart disease?"
docs_ss = retriever.get_relevant_documents(
    {"query": question, "filter": ""},
    k=5,
)

for doc in docs_ss:
    print(doc.page_content)
    print(doc.metadata)
print(len(docs_ss))

Philadelphia, PA WB Saunders.     Acute coronary syndromes
No ST elevation ST elevation
NSTEMI
Unstable angina NQ-MI QwMINo
enzyme
rise
{'page': 279, 'source': 'data/oxford-handbook-of-cardiology-2nbsped-9780199643219_compress.pdf'}
Following withdrawal due to hyperkalaemia, restart at a dose of 25 mg 
every other day if K  +  <5.0 mmol/L.        Table 2.6   Doses for commonly used aldosterone antagonists  
  Agent    Starting dose    Target dose  
 Spironolactone 
  Heart failure 
  Hypertension  25 mg daily 
 100 mg daily  50 mg daily a  
 25–200 mg daily 
 Eplerenone  25 mg daily  50 mg daily a  
      Table 2.7   Recommended dose adjustments based on potassium levels  
  Serum K  +   (mmol/L)    Action    Dose adjustment  
 <5.0  Increase dose  25 mg every other day to 25 mg daily 
 25 mg daily to 50 mg daily 
 5.0–5.4  Maintain dose  No dose adjustment 
 5.5–5.9  Decrease dose  50 mg daily to 25 mg daily 
 25 mg daily to 25 mg every other day 
 25 mg every other day to withhold dr

In [ ]:
# %%

# Proper query construction
content = "anonymous data from heart disease patients"
retriever = SelfQueryRetriever.from_llm(
    llm,
    diagnosis_db,
    content,
    metadata_field_info=metadata_field_info,
    embedding_function=ollama_embeddings,
)

# Example of a correct query
question = "What is the most common metric of each heart disease?"
docs_ss = retriever.get_relevant_documents(
    question,
    k=5,
)

for doc in docs_ss:
    print(doc.page_content)
    print(doc.metadata)
print(len(docs_ss))

Philadelphia, PA WB Saunders.     Acute coronary syndromes
No ST elevation ST elevation
NSTEMI
Unstable angina NQ-MI QwMINo
enzyme
rise
{'page': 279, 'source': 'data/oxford-handbook-of-cardiology-2nbsped-9780199643219_compress.pdf'}
Following withdrawal due to hyperkalaemia, restart at a dose of 25 mg 
every other day if K  +  <5.0 mmol/L.        Table 2.6   Doses for commonly used aldosterone antagonists  
  Agent    Starting dose    Target dose  
 Spironolactone 
  Heart failure 
  Hypertension  25 mg daily 
 100 mg daily  50 mg daily a  
 25–200 mg daily 
 Eplerenone  25 mg daily  50 mg daily a  
      Table 2.7   Recommended dose adjustments based on potassium levels  
  Serum K  +   (mmol/L)    Action    Dose adjustment  
 <5.0  Increase dose  25 mg every other day to 25 mg daily 
 25 mg daily to 50 mg daily 
 5.0–5.4  Maintain dose  No dose adjustment 
 5.5–5.9  Decrease dose  50 mg daily to 25 mg daily 
 25 mg daily to 25 mg every other day 
 25 mg every other day to withhold dr

In [ ]:
# %%

# Proper query construction
content = "anonymous data from heart disease patients"
retriever = SelfQueryRetriever.from_llm(
    llm,
    diagnosis_db,
    content,
    metadata_field_info=metadata_field_info,
    embedding_function=ollama_embeddings,
)

# Example of a correct query
question = "What is the most common metric of each heart disease?"
docs_ss = retriever.get_relevant_documents(
    question,
    k=5,
)

for doc in docs_ss:
    print(doc.page_content)
    print(doc.metadata)
print(len(docs_ss))

Philadelphia, PA WB Saunders.     Acute coronary syndromes
No ST elevation ST elevation
NSTEMI
Unstable angina NQ-MI QwMINo
enzyme
rise
{'page': 279, 'source': 'data/oxford-handbook-of-cardiology-2nbsped-9780199643219_compress.pdf'}
Following withdrawal due to hyperkalaemia, restart at a dose of 25 mg 
every other day if K  +  <5.0 mmol/L.        Table 2.6   Doses for commonly used aldosterone antagonists  
  Agent    Starting dose    Target dose  
 Spironolactone 
  Heart failure 
  Hypertension  25 mg daily 
 100 mg daily  50 mg daily a  
 25–200 mg daily 
 Eplerenone  25 mg daily  50 mg daily a  
      Table 2.7   Recommended dose adjustments based on potassium levels  
  Serum K  +   (mmol/L)    Action    Dose adjustment  
 <5.0  Increase dose  25 mg every other day to 25 mg daily 
 25 mg daily to 50 mg daily 
 5.0–5.4  Maintain dose  No dose adjustment 
 5.5–5.9  Decrease dose  50 mg daily to 25 mg daily 
 25 mg daily to 25 mg every other day 
 25 mg every other day to withhold dr

In [ ]:
# %%

# Proper query construction
content = "anonymous data from heart disease patients"
retriever = SelfQueryRetriever.from_llm(
    llm,
    diagnosis_db,
    content,
    metadata_field_info=metadata_field_info,
    embedding_function=ollama_embeddings,
    k = 100,
)

# Example of a correct query
question = "What is the most common metric of each heart disease?"
docs_ss = retriever.get_relevant_documents(
    question,
    k=10,
)

for doc in docs_ss:
    print(doc.page_content)
    print(doc.metadata)
print(len(docs_ss))

Philadelphia, PA WB Saunders.     Acute coronary syndromes
No ST elevation ST elevation
NSTEMI
Unstable angina NQ-MI QwMINo
enzyme
rise
{'page': 279, 'source': 'data/oxford-handbook-of-cardiology-2nbsped-9780199643219_compress.pdf'}
Following withdrawal due to hyperkalaemia, restart at a dose of 25 mg 
every other day if K  +  <5.0 mmol/L.        Table 2.6   Doses for commonly used aldosterone antagonists  
  Agent    Starting dose    Target dose  
 Spironolactone 
  Heart failure 
  Hypertension  25 mg daily 
 100 mg daily  50 mg daily a  
 25–200 mg daily 
 Eplerenone  25 mg daily  50 mg daily a  
      Table 2.7   Recommended dose adjustments based on potassium levels  
  Serum K  +   (mmol/L)    Action    Dose adjustment  
 <5.0  Increase dose  25 mg every other day to 25 mg daily 
 25 mg daily to 50 mg daily 
 5.0–5.4  Maintain dose  No dose adjustment 
 5.5–5.9  Decrease dose  50 mg daily to 25 mg daily 
 25 mg daily to 25 mg every other day 
 25 mg every other day to withhold dr